#  [모듈 3.1] 모델 튜닝 및 모델 등록 스텝 개발 (SageMaker Training Step)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 모델 튜닝 개요 
- 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 2. 모델 훈련 코드 확인
- 3. HPO 코드 실행
- 4. 모델 튜닝 스텝 개발 및 실행
    - 최적 성능 나오는 모델 등록
    
---

# 0. 모델 튜닝 개요

하이퍼파라미터 튜닝이라고도 하는 Amazon SageMaker 자동 모델 튜닝은 사용자가 지정한 알고리즘과 다양한 하이퍼파라미터를 사용하여 데이터 세트에 대해 여러 훈련 작업을 실행하여 최적의 모델 버전을 찾습니다. 그런 다음 선택한 지표로 측정된 값에 따라 최적의 성능을 보여준 모델을 만든 하이퍼파라미터 값을 선택합니다.



- 참고
    - 개발자 가이드: [SageMaker 로 자동 모델 튜닝 수행](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/automatic-model-tuning.html)
    - 공식 세이지 메이커의 샘플 입니다. --> [HPO 시작 코드](https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/xgboost_direct_marketing/hpo_xgboost_direct_marketing_sagemaker_python_sdk.ipynb)




# 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
---

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import boto3
import sagemaker
import pandas as pd
import os

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker")

%store -r 
# 노트북에 저장되어 있는 변수를 보기 위해서는 주석을 제거하고 실행하시면 됩니다.
# %store  

In [41]:
! aws s3 ls {train_preproc_data_uri} --recursive

2021-08-27 08:13:54     682602 sagemaker-pipeline-step-by-step-phase02/preporc/train.csv


In [42]:
train_prep_df = pd.read_csv(train_preproc_data_uri)
train_prep_df

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,8913.668763,80513.668763,54,94,0,1,750,3000,99207,...,0,0,1,0,0,0,1,0,1,0
1,0,19746.724395,26146.724395,41,165,0,1,750,2950,95632,...,0,0,0,1,0,0,0,1,0,1
2,0,11652.969918,22052.969918,57,155,0,1,750,3000,93203,...,0,0,1,0,0,0,0,1,0,1
3,0,11260.930936,115960.930936,39,80,0,1,750,3000,85208,...,0,0,1,0,0,0,1,0,1,0
4,0,27987.704652,31387.704652,39,60,0,1,750,3000,91792,...,0,1,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0,18052.611626,67152.611626,42,103,1,1,750,3000,93654,...,0,0,1,0,0,0,1,0,1,0
3996,0,34949.202468,51749.202468,23,6,0,3,750,3000,94305,...,0,0,0,1,1,0,0,0,1,0
3997,0,4063.701410,9963.701410,44,35,0,2,750,2550,95476,...,0,0,1,0,0,0,0,1,0,1
3998,0,17390.520451,20490.520451,22,38,0,1,750,3000,90680,...,0,1,0,0,0,0,0,1,0,1


## 2. 훈련 스크립트 확인
- 내용 확인을 위해서는 아래 주석을 제거하고 보세요
---

In [43]:
# !pygmentize src/xgboost_starter_script.py

## 3. HPO 코드 실행
---



### 기본 훈련 변수 및 하이퍼파라미터 설정

In [44]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = project_prefix

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
train_instance_count = 1

def get_pos_scale_weight(df, label):
    '''
    1, 0 의 레이블 분포를 계산하여 클래스 가중치 리턴
    예: 1: 10, 0: 90 이면 90/10 = 9 를 제공함. 
    호출:
        class_weight = get_pos_scale_weight(train_prep_df, label='fraud')
    '''
    fraud_sum = df[df[label] == 1].shape[0]
    non_fraud_sum = df[df[label] == 0].shape[0]
    class_weight = int(non_fraud_sum / fraud_sum)
    print(f"fraud_sum: {fraud_sum} , non_fraud_sum: {non_fraud_sum}, class_weight: {class_weight}")
    return class_weight
    
class_weight = get_pos_scale_weight(train_prep_df, label='fraud')

hyperparameters = {
       "scale_pos_weight" : class_weight,    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}


fraud_sum: 131 , non_fraud_sum: 3869, class_weight: 29


### 튜너 설정 및 생성
- xbg_estimator 정의된  estimator 기술
- `objective_metric_name = "validation:auc"` 튜닝을 하고자 하는 지표 기술
    - 이 지표의 경우는 훈련 코드에서 정의 및 기록을 해야만 합니다.
- `hyperparameter_ranges` 튜닝하고자 하는 파라미터의 범위 설정
- `max_jobs` 기술
    - 총 훈련잡의 갯수 입니다.
- `max_parallel_jobs` 기술
    - 병렬로 실행할 훈련잡의 개수 (리소스 제한에 따라서 에러가 발생할 수 있습니다. 이 경우에 줄여 주세요.)


In [45]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


xgb_estimator = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = 'ml.m4.2xlarge',
    framework_version = "1.0-1")

hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}

objective_metric_name = "validation:auc"

tuner = HyperparameterTuner(
    xgb_estimator, objective_metric_name, hyperparameter_ranges, 
    max_jobs=5,
    max_parallel_jobs=5,
)

In [46]:
tuner.fit(inputs = {'train': train_preproc_data_uri,
                   },
                  wait=False,
                 )


### 튜너의 실행 상태를 확인
- 약 5분 소요 됩니다.

In [47]:
import time

tuning_job_name = tuner.latest_tuning_job.job_name

def show_hpo_status(sm_client):
    status = sm_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuning_job_name
    )["HyperParameterTuningJobStatus"]
    return status

status = show_hpo_status(sm_client)
while status == 'InProgress':
    status = show_hpo_status(sm_client)    
    print("HPO status: ", status)    
    time.sleep(30)    

HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  InProgress
HPO status:  Completed


### Best 훈련 Job 출력
- 수행된 훈련 잡 중에서 가장 성능이 좋은 훈련 잡을 기술하고, 최종 사용된 하이퍼 파리미터 값을 보여 줌

In [48]:
from pprint import pprint

# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sm_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)
is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"] != "Maximize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["MetricName"]

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")


5 training jobs have completed
Best model found so far:
{'CreationTime': datetime.datetime(2021, 8, 27, 9, 9, 52, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.8271999955177307},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 8, 27, 9, 13, 18, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:028703291518:training-job/sagemaker-xgboost-210827-0909-005-d8ff35ec',
 'TrainingJobName': 'sagemaker-xgboost-210827-0909-005-d8ff35ec',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 8, 27, 9, 12, 19, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.557797354073632',
                          'eta': '0.2980081558006462',
                          'max_depth': '1',
                          'min_child_weight': '9.191874617446587'}}


### 튜닝을 수행한 모든 훈련 잡의 결과 확인
- `FinalObjectiveValue` 의 성능 지표 순서로 보여 줌

In [49]:
import pandas as pd

tuner_df = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner_df.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 5
{'lowest': 0.7914999723434448, 'highest': 0.8271999955177307}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,1.557797,0.298008,1.0,9.191875,sagemaker-xgboost-210827-0909-005-d8ff35ec,Completed,0.8272,2021-08-27 09:12:19+00:00,2021-08-27 09:13:18+00:00,59.0
2,1.891045,0.435041,1.0,8.057893,sagemaker-xgboost-210827-0909-003-c0747b20,Completed,0.8269,2021-08-27 09:12:44+00:00,2021-08-27 09:13:47+00:00,63.0
3,1.537592,0.085494,5.0,3.851098,sagemaker-xgboost-210827-0909-002-4df49fd6,Completed,0.8015,2021-08-27 09:12:12+00:00,2021-08-27 09:13:26+00:00,74.0
1,1.663407,0.881021,5.0,4.641075,sagemaker-xgboost-210827-0909-004-be0f0707,Completed,0.7977,2021-08-27 09:12:37+00:00,2021-08-27 09:13:46+00:00,69.0
4,0.879738,0.718337,6.0,6.042929,sagemaker-xgboost-210827-0909-001-291612d1,Completed,0.7915,2021-08-27 09:12:59+00:00,2021-08-27 09:14:11+00:00,72.0


# 4. 모델 튜닝 및 모델 등록 스텝 개발 및 실행
---
- 개발자 가이드의 튜닝 단계 참고 --> [튜닝 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning)



### 모델 빌딩 파이프라인 변수 생성



In [50]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

## (1) 튜닝 스텝 개발

### 튜너 기본 요소 정의
- 위에서 튜닝에 대한 부분을 다시 기술하였습니다. 값들을 바꾸어 가면서 사용하시면 됩니다. 

In [51]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)


xgb_train = XGBoost(
    entry_point = "xgboost_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = train_instance_count,
    instance_type = 'ml.m4.xlarge',
    framework_version = "1.0-1")

hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}

objective_metric_name = "validation:auc"

pipeline_tuner = HyperparameterTuner(
    xgb_train, 
    objective_metric_name, 
    hyperparameter_ranges, 
    max_jobs=5,
    max_parallel_jobs=5,
    #objective_type="Minimize",    
)

### 튜닝 단계 정의 



In [52]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.model import Model
    
step_tuning = TuningStep(
    name = "FraudTuning",
    tuner = pipeline_tuner,
    inputs={
        "train": TrainingInput(
            s3_data= train_preproc_data_uri,
            content_type="text/csv"
        ),
    },
)


## (2) 최적 모델 등록 스텝 개발

### 최고의 모델 생성 및 등록

하이퍼파라미터 튜닝 작업을 성공적으로 완료한 후TuningStep의 교육 작업에 의해 생성된 모델 아티팩트에서 SageMaker 모델을 생성하거나 모델 레지스트리에 모델을 등록할 수 있습니다.


TuningStep 클래스의 get_top_model_s3_uri 메서드를 사용하여 최고 성능의 모델 버전의 모델 아티팩트를 가져옵니다.

- `model_bucket_key` 는 튜닝 스텝을 통해서 생성된 훈련 잡의 위치 입니다.

In [53]:
model_bucket_key = estimator_output_path.split('//')[1]
print("model_bucket_key: ", model_bucket_key)
print("model_package_group_name: ", model_package_group_name)

model_bucket_key:  sagemaker-us-east-1-028703291518/sagemaker-pipeline-step-by-step-phase02/training_jobs
model_package_group_name:  sagemaker-pipeline-step-by-step-phase02


In [54]:
from sagemaker.workflow.step_collections import RegisterModel

step_register_best = RegisterModel(
    name="RegisterBestFraudModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_bucket_key),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)

### 모델 빌딩 파이프라인 정의

In [55]:
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig

pipeline_name = project_prefix + "-HPO-step"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        model_approval_status,        
    ],    
    pipeline_experiment_config=PipelineExperimentConfig(
      ExecutionVariables.PIPELINE_NAME,
      ExecutionVariables.PIPELINE_EXECUTION_ID
    ),    
    steps=[step_tuning, step_register_best],
    sagemaker_session=sagemaker_session,    
)

In [56]:
import json

definition = json.loads(pipeline.definition())
# definition

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [57]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


In [58]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:028703291518:pipeline/sagemaker-pipeline-step-by-step-phase02-hpo-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:028703291518:pipeline/sagemaker-pipeline-step-by-step-phase02-hpo-step/execution/db8kbmkdhxi1',
 'PipelineExecutionDisplayName': 'execution-1630055705362',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 8, 27, 9, 15, 5, 257000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 27, 9, 15, 5, 257000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '8f4d91a4-10f2-4c79-b0d5-fcd497530625',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8f4d91a4-10f2-4c79-b0d5-fcd497530625',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '461',
   'date': 'Fri, 27 Aug 2021 09:15:05 GMT'},
  'RetryAttempts': 0}}

In [59]:
execution.wait()

In [60]:
execution.list_steps()

[{'StepName': 'RegisterBestFraudModel',
  'StartTime': datetime.datetime(2021, 8, 27, 9, 20, 3, 42000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 27, 9, 20, 3, 847000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:028703291518:model-package/sagemaker-pipeline-step-by-step-phase02/3'}}},
 {'StepName': 'FraudTuning',
  'StartTime': datetime.datetime(2021, 8, 27, 9, 15, 6, 15000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 27, 9, 20, 2, 502000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TuningJob': {'Arn': 'arn:aws:sagemaker:us-east-1:028703291518:hyper-parameter-tuning-job/db8kbmkdhxi1-fraudtu-kzmzhfokk6'}}}]

# 5. 리소스 정리: 파이프라인
- 위에서 생성한 파이프라인을 제거 합니다.
- isDeletePipeline=False, verbose=Fasle
    - 파이프라인을 지우지 않고, 존재하는지 확인 합니다.
- isDeletePipeline=False, verbose=True
    - 파이프라인의 정의를 자세하 확인 합니다.
- isDeletePipeline=True, verbose=True or False
    - 파이프라인을 삭제 합니다.

In [61]:
from src.p_utils import clean_pipeline

# clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=True, verbose=False)   

pipeline sagemaker-pipeline-step-by-step-phase02-HPO-step exists
pipeline sagemaker-pipeline-step-by-step-phase02-HPO-step is deleted


## 참고: 에러 케이스 (RegisterBestAbaloneModel 스텝)
```
{'StepName': 'RegisterBestAbaloneModel',
  'StartTime': datetime.datetime(2021, 8, 21, 14, 18, 5, 830000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 21, 14, 18, 6, 305000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'ClientError: Cannot find S3 object: tuning-step-example/AbaloneTrain2222/4kk9kc5u8u7i-HPTunin-qR5ir5u32T-002-dbc3348a/output/model.tar.gz in bucket sagemaker-ap-northeast-2-057716757052. Please check if your S3 object exists and has proper permissions for SageMaker.',
  'Metadata': {}},
```

#### 해결
아래에서 `model_bucket_key` 의 경로가 step_tuning 을 통해서 training job의 모델 아티펙트 (model.tar.gz) 의 경로가 정확히 기술되어야 합니다. 
- [중요] 여기서 `s3://` 는 반드시 없어야 합니다.
    - 예: model_bucket_key:  sagemaker-ap-northeast-2-05771675/sagemaker-pipeline-step-by-step-phase01/training_jobs
    
    
```
step_register_best = RegisterModel(
    name="RegisterBestFraudModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0, s3_bucket=model_bucket_key),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
)
```